# Data processing

In [172]:
import boto3
import os
import re
import pickle
import json
import pgeocode

import pandas as pd
import numpy as np


from geopy.geocoders import Nominatim

from src.utils.general import get_s3_credentials

In [1]:
def read_pkl_from_s3(buket_name, filename):
    
    response = s3.get_object(
        Bucket=bucket_name,
        Key=filename)
    pkl_file = pickle.loads(response['Body'].read())
    return pkl_file

In [5]:
s3_creds = get_s3_credentials("../../conf/local/credentials.yaml")

INFO:root:Leyendo las credenciales de ../../conf/local/credentials.yaml


In [8]:
session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

In [11]:
bucket='data-product-architecture-equipo-3'

In [13]:
objects = s3.list_objects_v2(Bucket=bucket)['Contents']

In [22]:
for file in objects:
    print(file['Key'])

ingestion/consecutive/consecutive-inspections-2021-02-23.pkl
ingestion/initial/historic-inspections-2021-02-23.pkl


In [24]:
# Para obtener el path
os.path.dirname('ingestion/initial/historic-inspections-2021-02-23.pkl')

'ingestion/initial'

In [26]:
# Para obtener el nombre del archivo
filename = os.path.basename('ingestion/initial/historic-inspections-2021-02-23.pkl')
filename

'historic-inspections-2021-02-23.pkl'

In [35]:
# Para extraer la fecha del filename
re.findall(r'[ :]*\d+[ -/]\d+[ -/]\d+', filename, re.IGNORECASE)

['2021-02-23']

Por ahora sólo leeremos el inicial

In [130]:
filename="ingestion/initial/historic-inspections-2021-02-23.pkl"
print("Leyendo {}...".format(filename))
json_file = read_pkl_from_s3(bucket, filename)

Leyendo ingestion/initial/historic-inspections-2021-02-23.pkl...


In [137]:
# Convertimos a dataframe
food_inspection_df = pd.DataFrame(json_file)
food_inspection_df.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2484950,PHLOUR BAKERY AND CAFE,PHLOUR BAKERY AND CAFE,2522179,Bakery,Risk 1 (High),1138 W BRYN MAWR AVE,CHICAGO,IL,60660,2021-02-19T00:00:00.000,Canvass,Pass,40. PERSONAL CLEANLINESS - Comments: FOODHANDL...,41.98373915183195,-87.65964672940271,"{'latitude': '-87.65964672940271', 'longitude'..."
1,2484936,THE BAR ON BUENA,THE BAR ON BUENA,47876,Restaurant,Risk 1 (High),910 W BUENA AVE,CHICAGO,IL,60613,2021-02-19T00:00:00.000,Canvass,Fail,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,41.958550897849705,-87.65352004893188,"{'latitude': '-87.65352004893188', 'longitude'..."
2,2484930,EL PAISANO TACOS INC,EL PAISANO TACOS,2501058,Restaurant,Risk 1 (High),2429 W DIVISION ST,CHICAGO,IL,60622,2021-02-19T00:00:00.000,Complaint,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.90289443680921,-87.68823386858675,"{'latitude': '-87.68823386858675', 'longitude'..."
3,2484951,BELMONT CHILD CARE CENTER LLP,BELMONT CHILD CARE CENTER,2528209,Children's Services Facility,Risk 1 (High),5724-5728 W BELMONT AVE,CHICAGO,IL,60634,2021-02-19T00:00:00.000,License,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.93863961499259,-87.76992867781264,"{'latitude': '-87.76992867781264', 'longitude'..."
4,2484944,EDGEWATER MEXICAN CAFE,EDGEWATER MEXICAN CAFE,2615664,Restaurant,Risk 1 (High),1055 W BRYN MAWR AVE,CHICAGO,IL,60660,2021-02-19T00:00:00.000,Canvass,Pass,37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...,41.9835594257736,-87.65738202407279,"{'latitude': '-87.65738202407279', 'longitude'..."


# Pasamos todo el dataframe en minúsculas

In [138]:
# Poner todo en minúsculas

def df_to_lower_case(df):
    df = df.applymap(lambda s:s.lower() if type(s) == str else s)
    return df

food_inspection_df = df_to_lower_case(food_inspection_df)

In [84]:
def transformate_and_aggregate_dates(lista, df):
    df['inspection_date'] = pd.to_datetime(df['inspection_date'])
    
    date_gen = (getattr(df['inspection_date'].dt, i).rename(i) for i in lista)

    # concatenamos
    df = df.join(pd.concat(date_gen, axis=1))

    return df

In [139]:
# Extraer año, mes y día del inspection date
# Inspection date esta dado como año-mes-dia
food_inspection_df['inspection_date'] = pd.to_datetime(food_inspection_df['inspection_date'])

In [140]:
# Definimos la lista de lo que queremos
L = ['year', 'month', 'day', 'dayofweek', 'dayofyear', 'week', 'quarter']

date_gen = (getattr(food_inspection_df['inspection_date'].dt, i).rename(i) for i in L)

# concatenamos
food_inspection_df = food_inspection_df.join(pd.concat(date_gen, axis=1))

food_inspection_df

/home/user/.pyenv/versions/3.7.4/envs/food-inspection/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,...,latitude,longitude,location,year,month,day,dayofweek,dayofyear,week,quarter
0,2484950,phlour bakery and cafe,phlour bakery and cafe,2522179,bakery,risk 1 (high),1138 w bryn mawr ave,chicago,il,60660,...,41.98373915183195,-87.65964672940271,"{'latitude': '-87.65964672940271', 'longitude'...",2021,2,19,4,50,7,1
1,2484936,the bar on buena,the bar on buena,47876,restaurant,risk 1 (high),910 w buena ave,chicago,il,60613,...,41.958550897849705,-87.65352004893188,"{'latitude': '-87.65352004893188', 'longitude'...",2021,2,19,4,50,7,1
2,2484930,el paisano tacos inc,el paisano tacos,2501058,restaurant,risk 1 (high),2429 w division st,chicago,il,60622,...,41.90289443680921,-87.68823386858675,"{'latitude': '-87.68823386858675', 'longitude'...",2021,2,19,4,50,7,1
3,2484951,belmont child care center llp,belmont child care center,2528209,children's services facility,risk 1 (high),5724-5728 w belmont ave,chicago,il,60634,...,41.93863961499259,-87.76992867781264,"{'latitude': '-87.76992867781264', 'longitude'...",2021,2,19,4,50,7,1
4,2484944,edgewater mexican cafe,edgewater mexican cafe,2615664,restaurant,risk 1 (high),1055 w bryn mawr ave,chicago,il,60660,...,41.9835594257736,-87.65738202407279,"{'latitude': '-87.65738202407279', 'longitude'...",2021,2,19,4,50,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216516,104236,tempo cafe,tempo cafe,80916,restaurant,risk 1 (high),6 e chestnut st,chicago,il,60611,...,41.89843137207629,-87.6280091630558,"{'latitude': '-87.6280091630558', 'longitude':...",2010,1,4,0,4,1,1
216517,67732,wolcott's,troquet,1992039,restaurant,risk 1 (high),1834 w montrose ave,chicago,il,60613,...,41.961605669949854,-87.67596676683779,"{'latitude': '-87.67596676683779', 'longitude'...",2010,1,4,0,4,1,1
216518,67738,michael's on main cafe,michael's on main cafe,2008948,restaurant,risk 1 (high),8750 w bryn wawr ave,chicago,il,60631,...,NaN,NaN,NaN,2010,1,4,0,4,1,1
216519,52234,cafe 608,cafe 608,2013328,restaurant,risk 1 (high),608 w barry ave,chicago,il,60657,...,41.938006880423615,-87.6447545707008,"{'latitude': '-87.6447545707008', 'longitude':...",2010,1,4,0,4,1,1


In [141]:
food_inspection_df['city'].unique()

array(['chicago', nan, 'cchicago', 'chicagohicago', 'berwyn',
       'chicagochicago', 'evanston', '312chicago', 'chicagobedford park',
       'bridgeview', 'schaumburg', 'griffith', 'new york', 'elmhurst',
       'algonquin', 'new holstein', 'niles niles', 'chicago.', 'lansing',
       'wadsworth', 'wilmette', 'wheaton', 'rosemont', 'calumet city',
       'plainfield', 'highland park', 'palos park', 'elk grove village',
       'cicero', 'oak park', 'maywood', 'lake bluff', 'schiller park',
       'skokie', 'bannockburndeerfield', 'chcicago', 'bloomingdale',
       'norridge', 'charles a hayes', 'chchicago', 'chicagoi', 'summit',
       'oolympia fields', 'westmont', 'chicago heights', 'justice',
       'tinley park', 'lombard', 'east hazel crest', 'country club hills',
       'streamwood', 'bolingbrook', 'inactive', 'burnham', 'des plaines',
       'lake zurich', 'olympia fields', 'alsip', 'oak lawn',
       'blue island', 'glencoe', 'frankfort', 'naperville', 'broadview',
       'wor

In [142]:
# No es la mejor forma pero ..mientras :3
def change_misspelled_chicago_city_names(df):
    lista_misspelled = ['cchicago', 'chicago.', 'chicagohicago', 'chicagochicago', '312chicago', 'chicago.', 
                        'chcicago', 'chchicago', 'chicagoi']
    
    for name in lista_misspelled:
        df.loc[df['city'] == name, 'city'] = 'chicago'
        
    df.loc[df['city'] == 'oolympia fields', 'city'] = 'olympia fields'
    
    return df

In [143]:
food_inspection_df = change_misspelled_chicago_city_names(food_inspection_df)
food_inspection_df['city'].unique()

array(['chicago', nan, 'berwyn', 'evanston', 'chicagobedford park',
       'bridgeview', 'schaumburg', 'griffith', 'new york', 'elmhurst',
       'algonquin', 'new holstein', 'niles niles', 'lansing', 'wadsworth',
       'wilmette', 'wheaton', 'rosemont', 'calumet city', 'plainfield',
       'highland park', 'palos park', 'elk grove village', 'cicero',
       'oak park', 'maywood', 'lake bluff', 'schiller park', 'skokie',
       'bannockburndeerfield', 'bloomingdale', 'norridge',
       'charles a hayes', 'summit', 'olympia fields', 'westmont',
       'chicago heights', 'justice', 'tinley park', 'lombard',
       'east hazel crest', 'country club hills', 'streamwood',
       'bolingbrook', 'inactive', 'burnham', 'des plaines', 'lake zurich',
       'alsip', 'oak lawn', 'blue island', 'glencoe', 'frankfort',
       'naperville', 'broadview', 'worth', 'evergreen park'], dtype=object)

In [144]:
food_inspection_df.columns

Index(['inspection_id', 'dba_name', 'aka_name', 'license_', 'facility_type',
       'risk', 'address', 'city', 'state', 'zip', 'inspection_date',
       'inspection_type', 'results', 'violations', 'latitude', 'longitude',
       'location', 'year', 'month', 'day', 'dayofweek', 'dayofyear', 'week',
       'quarter'],
      dtype='object')

In [145]:
food_inspection_df.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,...,latitude,longitude,location,year,month,day,dayofweek,dayofyear,week,quarter
0,2484950,phlour bakery and cafe,phlour bakery and cafe,2522179,bakery,risk 1 (high),1138 w bryn mawr ave,chicago,il,60660,...,41.98373915183195,-87.65964672940271,"{'latitude': '-87.65964672940271', 'longitude'...",2021,2,19,4,50,7,1
1,2484936,the bar on buena,the bar on buena,47876,restaurant,risk 1 (high),910 w buena ave,chicago,il,60613,...,41.958550897849705,-87.65352004893188,"{'latitude': '-87.65352004893188', 'longitude'...",2021,2,19,4,50,7,1
2,2484930,el paisano tacos inc,el paisano tacos,2501058,restaurant,risk 1 (high),2429 w division st,chicago,il,60622,...,41.90289443680921,-87.68823386858675,"{'latitude': '-87.68823386858675', 'longitude'...",2021,2,19,4,50,7,1
3,2484951,belmont child care center llp,belmont child care center,2528209,children's services facility,risk 1 (high),5724-5728 w belmont ave,chicago,il,60634,...,41.93863961499259,-87.76992867781264,"{'latitude': '-87.76992867781264', 'longitude'...",2021,2,19,4,50,7,1
4,2484944,edgewater mexican cafe,edgewater mexican cafe,2615664,restaurant,risk 1 (high),1055 w bryn mawr ave,chicago,il,60660,...,41.9835594257736,-87.65738202407279,"{'latitude': '-87.65738202407279', 'longitude'...",2021,2,19,4,50,7,1


In [146]:
def remove_non_useful_columns(df):
    non_useful_cols = ['location', 'inspection_date', 'inspection_id', 'dba_name', 'aka_name', 'longitude', 
                       'latitude']
    df = df.drop(non_useful_cols, axis=1)
    
    return df

In [105]:
# Por ahora no las eliminaremos
# food_inspection_df = remove_non_useful_columns(food_inspection_df)

In [106]:
food_inspection_df.head()

,license_,facility_type,risk,address,city,state,zip,inspection_type,results,violations,latitude,longitude,year,month,day,dayofweek,dayofyear,weekofyear,quarter
0,2522179,bakery,risk 1 (high),1138 w bryn mawr ave,chicago,il,60660,canvass,pass,40. personal cleanliness - comments: foodhandl...,41.98373915183195,-87.65964672940271,2021,2,19,4,50,7,1
1,47876,restaurant,risk 1 (high),910 w buena ave,chicago,il,60613,canvass,fail,39. contamination prevented during food prepar...,41.958550897849705,-87.65352004893188,2021,2,19,4,50,7,1
2,2501058,restaurant,risk 1 (high),2429 w division st,chicago,il,60622,complaint,pass,47. food & non-food contact surfaces cleanable...,41.90289443680921,-87.68823386858675,2021,2,19,4,50,7,1
3,2528209,children's services facility,risk 1 (high),5724-5728 w belmont ave,chicago,il,60634,license,fail,10. adequate handwashing sinks properly suppli...,41.93863961499259,-87.76992867781264,2021,2,19,4,50,7,1
4,2615664,restaurant,risk 1 (high),1055 w bryn mawr ave,chicago,il,60660,canvass,pass,37. food properly labeled; original container ...,41.9835594257736,-87.65738202407279,2021,2,19,4,50,7,1


# Buscamos dónde hay nulos y los imputamos 

In [147]:
columna = 'license_'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

17

# Facility type

In [148]:
columna = 'facility_type'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

4891

In [197]:
# imputamos facility type con Unkown
food_inspection_df.loc[bool_series, 'facility_type'] = "unkown"


# Risk

In [202]:
columna = 'risk'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

69

In [209]:
# Cambiamos nombres
food_inspection_df.loc[food_inspection_df['risk'] == 'risk 1 (high)', 'risk'] = "high"
food_inspection_df.loc[food_inspection_df['risk'] == 'risk 2 (medium)', 'risk'] = "medium"
food_inspection_df.loc[food_inspection_df['risk'] == 'risk 3 (low)', 'risk'] = "low"
food_inspection_df.loc[bool_series, 'risk'] = 'unkown_risk'

# Address

In [150]:
columna = 'address'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

0

In [228]:
food_inspection_df['address'][0].strip().split(' ')[-1]

'ave'

In [242]:
food_inspection_df['street_type'] = 'unknown street type'

In [234]:
food_inspection_df.street_type = food_inspection_df.applymap(lambda s:s.strip().split(' ')[-1] if type(s) == str else s)

In [250]:
# Esto es mejor hacerlo con un apply
for i in range(0,len(food_inspection_df)):
    food_inspection_df.loc[i, 'street_type'] = food_inspection_df.loc[i, 'address'].strip().split(' ')[-1]

In [251]:
food_inspection_df.street_type.unique()

array(['ave', 'st', 'rd', 'hwy', 'dr', 'blvd', 'bldg', 'broadway',
       '(1e&w)', 'pkwy', '(6200n)', 'cicero', '1st', 'market', 'pl',
       'belmont', 'sheridan', 'fl', 'milwaukee', 'e', 'lawrence', 'byron',
       '(432w)', '1', 'a', 'ste1', '(3332s)', 'park', 'wells', 'chicago',
       '(1700w)', 'halsted', 'dorchester', 'division', 'narragansett',
       '(635e)', 'clybourn', 'cermak', '(1232w)', '(8700s)', '(45e)',
       'langley', 'kenwood', '(1900w)', 'o', 'ravenswood', 'plz', 'west',
       '(2200w)', '(332w)', 'devon', '(632w)', '(1300w)', '(300w)',
       'clinton', '(6032w)', '(600s)', '(600w)', 'clark', '(1132e)',
       '(140w)', '(832w)', 'fullerton', 'franklin', '(100n)', 'ct', 'c',
       '(100e)', '(1600e)', '(1432w)', 'lamon', '(344e)', '(1500w)',
       '(1700e)', 'madison', '(200e)', 'ingleside', '(700e)',
       'paulina(1700w)', '(700w)', 'homan', '(8000s)', 'indiana',
       '(1732w)', 'wood', 'st(5100s)', '(2800e)', '(600n)', 'phillips',
       'marshfield(16

In [151]:
columna = 'city'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

161

In [152]:
columna = 'state'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

50

# ZIP

In [170]:
columna = 'zip'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])
    
# filtering data  
# displayind data only with license = Not NaN  
print(len(food_inspection_df[bool_series]))
food_inspection_df[bool_series].head()

46


,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,...,latitude,longitude,location,year,month,day,dayofweek,dayofyear,week,quarter
118,2484790,lux galerie,lux galerie,2771243,restaurant,risk 1 (high),4012 s western ave,NaN,il,NaN,...,41.820691707442,-87.68490231909283,"{'latitude': '-87.68490231909283', 'longitude'...",2021,2,11,3,42,6,1
352,2484503,lux galerie,lux galerie,2771243,restaurant,risk 1 (high),4012 s western ave,NaN,il,NaN,...,41.820691707442,-87.68490231909283,"{'latitude': '-87.68490231909283', 'longitude'...",2021,2,4,3,35,5,1
4511,2453344,tacos cincuenta arrobas,tacos cincuenta arrobas,2703817,restaurant,risk 2 (medium),2300 s throop st,chicago,il,NaN,...,41.85045102427,-87.65879785567869,"{'latitude': '-87.65879785567869', 'longitude'...",2020,10,14,2,288,42,4
40036,2222692,leo's food and liquor,leo's liquor,2535068,liquor,risk 3 (low),4471 w lawrence ave,NaN,il,NaN,...,41.96799335620968,-87.74072634914224,"{'latitude': '-87.74072634914224', 'longitude'...",2018,9,21,4,264,38,3
42358,2200428,leo's food and liquor,leo's liquor,2535068,liquor,risk 3 (low),4471 w lawrence ave,NaN,il,NaN,...,41.96799335620968,-87.74072634914224,"{'latitude': '-87.74072634914224', 'longitude'...",2018,8,2,3,214,31,3


In [169]:
columna = 'zip'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  & pd.isnull(food_inspection_df['location'])
    
# filtering data  
# displayind data only with license = Not NaN  
print(len(food_inspection_df[bool_series]))
food_inspection_df[bool_series]

3


,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,...,latitude,longitude,location,year,month,day,dayofweek,dayofyear,week,quarter
91179,1763245,eat n run chicken and fish,eat n run chicken and fish,2442868,restaurant,risk 1 (high),,NaN,il,NaN,...,NaN,NaN,NaN,2016,4,19,1,110,16,2
145488,1106878,filipino american council,rizal center,2225427,golden diner,risk 1 (high),1332 w driving park road bsmt,NaN,il,NaN,...,NaN,NaN,NaN,2013,9,26,3,269,39,3
212213,60405,casa central la posada,casa central la posada,0,restaurant,risk 1 (high),,NaN,il,NaN,...,NaN,NaN,NaN,2010,4,5,0,95,14,2


In [180]:
columna = 'zip'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  & pd.notnull(food_inspection_df['location'])
    
# filtering data  
# displayind data only with license = Not NaN  
print(len(food_inspection_df[bool_series]))
zip_notnull_df = food_inspection_df[bool_series]

43


In [181]:
def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']['postcode']


geolocator = Nominatim(user_agent='http' )

In [182]:
zipcodes = zip_notnull_df.apply(get_zipcode, axis=1, 
                                    geolocator=geolocator, 
                                    lat_field='latitude', lon_field='longitude')

118       60643
352       60643
4511      60608
40036     60630
42358     60630
51776     60616
106181    60614
106426    60614
111296    60659
115713    60616
118667    60640
122515    60606
135309    60626
137973    60654
138283    60654
142240    60616
142344    60616
142371    60619
145877    60605
146029    60605
146047    60605
156729    60608
156806    60616
161282    60608
169128    60610
174980    60626
177272    60626
185901    60634
190983    60605
191043    60605
191084    60605
195147    60640
195716    60640
196257    60640
196376    60640
196478    60640
196833    60640
197385    60640
198852    60636
201453    60649
203969    60606
214012    60623
216243    60623
dtype: object

In [187]:
food_inspection_df.loc[bool_series,'zip'] = zipcodes

# Inspection type

In [154]:
columna = 'inspection_type'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

1

In [155]:
columna = 'results'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

0

In [156]:
columna = 'violations'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

57856

# Location and latitude

In [157]:
columna = 'latitude'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

716

In [158]:
columna = 'longitude'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna])  
    
# filtering data  
# displayind data only with license = Not NaN  
len(food_inspection_df[bool_series])

716

In [165]:
columna = 'location'

# creating bool series True for NaN values  
bool_series = pd.isnull(food_inspection_df[columna]) & pd.isnull(food_inspection_df['latitude']) & pd.isnull(food_inspection_df['longitude'] )
    
# filtering data  
# displayind data only with license = Not NaN  
print(len(food_inspection_df[bool_series]))
food_inspection_df[bool_series]

716


,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,...,latitude,longitude,location,year,month,day,dayofweek,dayofyear,week,quarter
852,2473009,jeni's splendid ice creams,jeni's splendid ice creams,2767260,restaurant,risk 2 (medium),2512 n milkwaukee ave,chicago,il,60647,...,NaN,NaN,NaN,2021,1,22,4,22,3,1
869,2472956,2096 n milwaukee llc,2096 n milwaukee llc,2767033,restaurant,risk 1 (high),2096 n milkwaukee ave,chicago,il,60647,...,NaN,NaN,NaN,2021,1,21,3,21,3,1
876,2472982,enzo & emilia,enzo & emilia,2762590,restaurant,risk 1 (high),2840-2542 n milkwaukee ave,chicago,il,60647,...,NaN,NaN,NaN,2021,1,21,3,21,3,1
882,2472978,enzo & emilia,enzo & emilia,2762588,restaurant,risk 1 (high),2840-2542 n milkwaukee ave,chicago,il,60647,...,NaN,NaN,NaN,2021,1,21,3,21,3,1
897,2472938,2096 n milwaukee llc,2096 n milwaukee llc,2767032,restaurant,risk 1 (high),2096 n milkwaukee ave,chicago,il,60647,...,NaN,NaN,NaN,2021,1,21,3,21,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216106,67840,tonti elementary,tonti elementary,25631,school,risk 1 (high),5815 s homan (3400w) ave,chicago,il,60629,...,NaN,NaN,NaN,2010,1,12,1,12,2,1
216189,158253,argentina foods,argentina foods,2009367,grocery store,risk 2 (medium),4500 s woods st,chicago,il,60609,...,NaN,NaN,NaN,2010,1,11,0,11,2,1
216302,67788,carson (old),carson (old),22601,school,risk 1 (high),5516 s maplewood (2532w) ave,chicago,il,60629,...,NaN,NaN,NaN,2010,1,7,3,7,1,1
216331,154225,chartwells,chartwells,1166638,special event,risk 2 (medium),2324 n fremont st,chicago,il,60614,...,NaN,NaN,NaN,2010,1,7,3,7,1,1
